# EDA UPDATTE

In [1]:
import os
import pandas as pd
import re  # Importar el módulo para trabajar con expresiones regulares
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows

from config import ruta
from config import ruta_archivo_entregable

ModuleNotFoundError: No module named 'config'

In [ ]:
# Listar todos los archivos en el directorio
archivos = os.listdir(ruta)

# Filtrar y procesar solo los archivos que terminan en .csv
print("Procesando archivos CSV:")
for archivo in archivos:
    if archivo.endswith('.csv'):
        # Construir la ruta completa al archivo
        archivo_completo = os.path.join(ruta, archivo)
        # Leer el archivo CSV
        df = pd.read_csv(archivo_completo, encoding='utf-8')
        # Mostrar la cantidad de filas
        print(f"El archivo {archivo} tiene {df.shape[0]} filas.")
        # Reemplazar puntos por comas en la columna 'Duración'
        if 'Duración' in df.columns:
            df['Duración'] = df['Duración'].astype(str).str.replace('.', ',')
            # Guardar el archivo con los cambios
            df.to_csv(archivo_completo, index=False, encoding='utf-8')
            print(f"Archivo procesado: {archivo}")
        else:
            print(f"La columna 'Duración' no existe en el archivo: {archivo}")


Procesando archivos CSV:
El archivo ACTIVIDADES 0002_GASCOL CENTRO (1).csv tiene 57 filas.
Archivo procesado: ACTIVIDADES 0002_GASCOL CENTRO (1).csv
El archivo ACTIVIDADES 0011_GASCOL SUR (3).csv tiene 310 filas.
Archivo procesado: ACTIVIDADES 0011_GASCOL SUR (3).csv
El archivo ACTIVIDADES 0021_G. LUX BOGOTÁ (1).csv tiene 693 filas.
Archivo procesado: ACTIVIDADES 0021_G. LUX BOGOTÁ (1).csv
El archivo ACTIVIDADES 0079_BELLO (1).csv tiene 718 filas.
Archivo procesado: ACTIVIDADES 0079_BELLO (1).csv
El archivo ACTIVIDADES 0092_MEDELLÍN (2).csv tiene 260 filas.
Archivo procesado: ACTIVIDADES 0092_MEDELLÍN (2).csv
El archivo ACTIVIDADES 0111_MALAMBO (1).csv tiene 875 filas.
Archivo procesado: ACTIVIDADES 0111_MALAMBO (1).csv
El archivo ACTIVIDADES 0198_POSTOBÓN PEREIRA (1).csv tiene 64 filas.
Archivo procesado: ACTIVIDADES 0198_POSTOBÓN PEREIRA (1).csv
El archivo ACTIVIDADES 0381_G. LUX MONTERÍA (1).csv tiene 64 filas.
Archivo procesado: ACTIVIDADES 0381_G. LUX MONTERÍA (1).csv
El archivo A

In [ ]:
# Listar todos los archivos en el directorio
archivos = os.listdir(ruta)

# Filtrar solo los archivos que terminan en .csv
archivos_csv = [archivo for archivo in archivos if archivo.endswith('.csv')]

# Función para extraer el código de cuatro dígitos
def extraer_codigo(nombre_archivo):
    match = re.search(r'\d{4}', nombre_archivo)
    return match.group(0) if match else 'Código no encontrado'

# Función para calcular la sumatoria de la columna "Duración" en cada archivo CSV
def sumar_duracion(nombre_archivo):
    try:
        # Cargar el archivo CSV
        df = pd.read_csv(os.path.join(ruta, nombre_archivo))
        # Reemplazar comas por puntos y convertir a flotante de forma segura
        df['Duración'] = pd.to_numeric(df['Duración'].str.replace(',', '.'), errors='coerce')
        # Sumar la columna "Duración", ignorando NaNs y valores no numéricos
        suma_duracion = df['Duración'].dropna().sum()
        return suma_duracion
    except Exception as e:
        return f"Error: {e}"

# Función para sumar "Duración" donde la columna "Nombre" tiene algún dato o usar "codigo_personal_text"
def sumar_duracion_planeada(nombre_archivo):
    try:
        # Cargar el archivo CSV
        df = pd.read_csv(os.path.join(ruta, nombre_archivo))
        # Determinar qué columna usar ("Nombre" o "codigo_personal_text")
        if 'Nombre' in df.columns and df['Nombre'].notna().any():
            columna_usada = 'Nombre'
        elif 'codigo_personal_text' in df.columns and df['codigo_personal_text'].notna().any():
            columna_usada = 'codigo_personal_text'
        else:
            return 'Columna relevante no encontrada'
        # Reemplazar comas por puntos y convertir a flotante de forma segura
        df['Duración'] = pd.to_numeric(df['Duración'].str.replace(',', '.'), errors='coerce')
        # Filtrar filas donde la columna seleccionada tiene algún dato no nulo y no vacío
        df_filtrado = df[df[columna_usada].notna() & df[columna_usada].str.strip().astype(bool)]
        # Sumar la "Duración" filtrada, ignorando NaNs y valores no numéricos
        suma_duracion_planeada = df_filtrado['Duración'].dropna().sum()
        return suma_duracion_planeada
    except Exception as e:
        return f"Error: {e}"

# Función para sumar "Duración" donde la columna "Avance" o "Completado" tiene algún dato
def sumar_duracion_avance(nombre_archivo):
    try:
        # Cargar el archivo CSV
        df = pd.read_csv(os.path.join(ruta, nombre_archivo))
        # Reemplazar comas por puntos y convertir a flotante de forma segura
        df['Duración'] = pd.to_numeric(df['Duración'].str.replace(',', '.'), errors='coerce')
        # Intentar filtrar filas donde la columna "Avance" tiene algún dato no nulo y no vacío
        if 'Avance' in df.columns:
            df_filtrado = df[df['Avance'].notna() & df['Avance'].astype(str).str.strip().astype(bool)]
        # Si no se encuentra la columna "Avance", usar la columna "Completado"
        elif 'Completado' in df.columns:
            df_filtrado = df[df['Completado'].notna() & df['Completado'].astype(str).str.strip().astype(bool)]
        else:
            return 'No se encontraron columnas relevantes ("Avance" o "Completado")'
        # Sumar la "Duración" filtrada, ignorando NaNs y valores no numéricos
        suma_duracion_avance = df_filtrado['Duración'].dropna().sum()
        return suma_duracion_avance
    except Exception as e:
        return f"Error: {e}"

def duracion_tipo_o_total(nombre_archivo):
    try:
        # Cargar el archivo CSV
        df = pd.read_csv(os.path.join(ruta, nombre_archivo))
        # Reemplazar comas por puntos y convertir a flotante de forma segura
        df['Duración'] = pd.to_numeric(df['Duración'].str.replace(',', '.'), errors='coerce')
        
        # Si la columna 'Tipo' existe y tiene al menos un dato no nulo
        if 'Tipo' in df.columns:
            # Filtrar filas donde la columna 'Tipo' tiene algún dato no nulo y no vacío
            df_filtrado = df[df['Tipo'].notna() & df['Tipo'].str.strip().astype(bool)]
            # Sumar la columna "Duración" filtrada, ignorando NaNs y valores no numéricos
            suma_duracion = df_filtrado['Duración'].dropna().sum()
        else:
            # Si la columna 'Tipo' no existe, sumar toda la columna "Duración"
            suma_duracion = df['Duración'].dropna().sum()

        return suma_duracion
    except Exception as e:
        return f"Error: {e}"
    
def duracion_tipo_preventivo_o_total(nombre_archivo):
    try:
        # Cargar el archivo CSV
        df = pd.read_csv(os.path.join(ruta, nombre_archivo))
        # Reemplazar comas por puntos y convertir a flotante de forma segura
        df['Duración'] = pd.to_numeric(df['Duración'].str.replace(',', '.'), errors='coerce')
        
        # Si la columna 'Tipo' existe
        if 'Tipo' in df.columns:
            # Filtrar filas donde la columna 'Tipo' es igual a 'Preventivo'
            df_filtrado = df[df['Tipo'].notna() & (df['Tipo'] == 'Preventivo')]
            # Sumar la columna "Duración" filtrada, ignorando NaNs y valores no numéricos
            suma_duracion = df_filtrado['Duración'].dropna().sum()
            return suma_duracion if suma_duracion > 0 else "Sin datos 'Preventivo'"
        else:
            # Si la columna 'Tipo' no existe, sumar toda la columna "Duración"
            suma_duracion_total = df['Duración'].dropna().sum()
            return suma_duracion_total if suma_duracion_total > 0 else "Sin datos de duración"
    except Exception as e:
        return f"Error: {e}"

def duracion_tipo_preventivo_nombre_codigo(nombre_archivo):
    try:
        # Cargar el archivo CSV
        df = pd.read_csv(os.path.join(ruta, nombre_archivo))
        # Reemplazar comas por puntos y convertir a flotante de forma segura
        df['Duración'] = pd.to_numeric(df['Duración'].str.replace(',', '.'), errors='coerce')

        # Verificar las columnas necesarias
        columna_nombre = 'Nombre' if 'Nombre' in df.columns else ('codigo_personal_text' if 'codigo_personal_text' in df.columns else None)

        # Si no existe la columna 'Tipo', llamar a sumar_duracion_planeada
        if 'Tipo' not in df.columns:
            return sumar_duracion_planeada(nombre_archivo)

        # Si existen las columnas 'Tipo' y alguna columna relevante de nombre o código
        if columna_nombre:
            # Filtrar filas donde la columna 'Tipo' es igual a 'Preventivo' y la otra columna tiene algún dato
            df_filtrado = df[df['Tipo'].notna() & (df['Tipo'] == 'Preventivo') & df[columna_nombre].notna() & df[columna_nombre].str.strip().astype(bool)]
            # Sumar la columna "Duración" filtrada, ignorando NaNs y valores no numéricos
            suma_duracion = df_filtrado['Duración'].dropna().sum()
            return suma_duracion if suma_duracion > 0 else "Sin datos 'Preventivo' con datos relevantes en columna"
        
        # Si la columna 'Tipo' existe pero no la columna relevante
        return "Columna relevante ('Nombre'/'codigo_personal_text') no encontrada"

    except Exception as e:
        return f"Error al procesar el archivo: {e}"

def duracion_tipo_preventivo_avance(nombre_archivo):
    try:
        # Cargar el archivo CSV
        df = pd.read_csv(os.path.join(ruta, nombre_archivo))
        # Reemplazar comas por puntos y convertir a flotante de forma segura
        df['Duración'] = pd.to_numeric(df['Duración'].str.replace(',', '.'), errors='coerce')

        # Si la columna 'Tipo' existe y es adecuada para el filtrado
        if 'Tipo' in df.columns:
            # Filtrar filas donde la columna 'Tipo' es igual a 'Preventivo'
            df_filtrado = df[df['Tipo'].notna() & (df['Tipo'] == 'Preventivo')]
            # Calcular la suma de las duraciones para las filas filtradas
            suma_duracion = df_filtrado['Duración'].dropna().sum()
            # Obtener la cantidad de filas filtradas que contienen 'Preventivo'
            cantidad_filas_preventivo = len(df_filtrado)
            # Verificar si hay datos para retornar
            if cantidad_filas_preventivo > 0:
                # Calcular promedio de duración por cada fila 'Preventivo'
                promedio_duracion = suma_duracion / cantidad_filas_preventivo
                return promedio_duracion
            else:
                return "Sin datos 'Preventivo'"
        else:
            # Si la columna 'Tipo' no existe, retornar el resultado de otra función de manejo
            return sumar_duracion_avance(nombre_archivo)
    except Exception as e:
        return f"Error: {e}"

    
def duracion_tipo_predictivo_o_total(nombre_archivo):
    try:
        # Cargar el archivo CSV
        df = pd.read_csv(os.path.join(ruta, nombre_archivo))
        # Reemplazar comas por puntos y convertir a flotante de forma segura
        df['Duración'] = pd.to_numeric(df['Duración'].str.replace(',', '.'), errors='coerce')
        
        # Si la columna 'Tipo' existe
        if 'Tipo' in df.columns:
            # Filtrar filas donde la columna 'Tipo' es igual a 'Predictivo'
            df_filtrado = df[df['Tipo'].notna() & (df['Tipo'] == 'Predictivo')]
            # Sumar la columna "Duración" filtrada, ignorando NaNs y valores no numéricos
            suma_duracion = df_filtrado['Duración'].dropna().sum()
            return suma_duracion if suma_duracion > 0 else "Sin datos 'Predictivo'"
        else:
            # Si la columna 'Tipo' no existe, marcar valor 0
            return 0
    except Exception as e:
        return f"Error: {e}"
    
def duracion_tipo_predictivo_nombre_codigo(nombre_archivo):
    try:
        # Cargar el archivo CSV
        df = pd.read_csv(os.path.join(ruta, nombre_archivo))
        # Reemplazar comas por puntos y convertir a flotante de forma segura
        df['Duración'] = pd.to_numeric(df['Duración'].str.replace(',', '.'), errors='coerce')

        # Verificar las columnas necesarias
        columna_nombre = 'Nombre' if 'Nombre' in df.columns else ('codigo_personal_text' if 'codigo_personal_text' in df.columns else None)

        # Si no existe la columna 'Tipo', retornar 0
        if 'Tipo' not in df.columns:
            return 0  # Cambio hecho aquí: retornar 0 directamente

        # Si existen las columnas 'Tipo' y alguna columna relevante de nombre o código
        if columna_nombre:
            # Filtrar filas donde la columna 'Tipo' es igual a 'Predictivo' y la otra columna tiene algún dato
            df_filtrado = df[df['Tipo'].notna() & (df['Tipo'] == 'Predictivo') & df[columna_nombre].notna() & df[columna_nombre].str.strip().astype(bool)]
            # Sumar la columna "Duración" filtrada, ignorando NaNs y valores no numéricos
            suma_duracion = df_filtrado['Duración'].dropna().sum()
            return suma_duracion if suma_duracion > 0 else "Sin datos 'Predictivo' con datos relevantes en columna"
        
        # Si la columna 'Tipo' existe pero no la columna relevante
        return "Columna relevante ('Nombre'/'codigo_personal_text') no encontrada"

    except Exception as e:
        return f"Error al procesar el archivo: {e}"
    
def duracion_tipo_predictivo_avance(nombre_archivo):
    try:
        # Cargar el archivo CSV
        df = pd.read_csv(os.path.join(ruta, nombre_archivo))
        # Reemplazar comas por puntos y convertir a flotante de forma segura
        df['Duración'] = pd.to_numeric(df['Duración'].str.replace(',', '.'), errors='coerce')

        # Verificar si la columna 'Tipo' existe y es adecuada para el filtrado
        if 'Tipo' in df.columns:
            # Filtrar filas donde la columna 'Tipo' es igual a 'Predictivo'
            df_filtrado = df[df['Tipo'].notna() & (df['Tipo'] == 'Predictivo')]
            # Calcular la suma de las duraciones para las filas filtradas
            suma_duracion = df_filtrado['Duración'].dropna().sum()
            # Obtener la cantidad de filas filtradas que contienen 'Predictivo'
            cantidad_filas_predictivo = len(df_filtrado)
            # Verificar si hay datos para retornar
            if cantidad_filas_predictivo > 0:
                # Calcular promedio de duración por cada fila 'Predictivo'
                promedio_duracion = suma_duracion / cantidad_filas_predictivo
                return promedio_duracion
            else:
                return "Sin datos 'Predictivo'"
        else:
            # Si la columna 'Tipo' no existe, retornar cero
            return 0
    except Exception as e:
        return f"Error: {e}"
    
    
def duracion_tipo_correctivo_o_total(nombre_archivo):
    try:
        # Cargar el archivo CSV
        df = pd.read_csv(os.path.join(ruta, nombre_archivo))
        # Reemplazar comas por puntos y convertir a flotante de forma segura
        df['Duración'] = pd.to_numeric(df['Duración'].str.replace(',', '.'), errors='coerce')
        
        # Si la columna 'Tipo' existe
        if 'Tipo' in df.columns:
            # Filtrar filas donde la columna 'Tipo' es igual a 'Correctivo'
            df_filtrado = df[df['Tipo'].notna() & (df['Tipo'] == 'Correctivo')]
            # Sumar la columna "Duración" filtrada, ignorando NaNs y valores no numéricos
            suma_duracion = df_filtrado['Duración'].dropna().sum()
            return suma_duracion if suma_duracion > 0 else "Sin datos 'Correctivo'"
        else:
            # Si la columna 'Tipo' no existe, marcar valor 0
            return 0
    except Exception as e:
        return f"Error: {e}"
    
def duracion_tipo_correctivo_nombre_codigo(nombre_archivo):
    try:
        # Cargar el archivo CSV
        df = pd.read_csv(os.path.join(ruta, nombre_archivo))
        # Reemplazar comas por puntos y convertir a flotante de forma segura
        df['Duración'] = pd.to_numeric(df['Duración'].str.replace(',', '.'), errors='coerce')

        # Verificar las columnas necesarias
        columna_nombre = 'Nombre' if 'Nombre' in df.columns else ('codigo_personal_text' if 'codigo_personal_text' in df.columns else None)

        # Si no existe la columna 'Tipo', retornar 0
        if 'Tipo' not in df.columns:
            return 0

        # Si existen las columnas 'Tipo' y alguna columna relevante de nombre o código
        if columna_nombre:
            # Filtrar filas donde la columna 'Tipo' es igual a 'Correctivo' y la otra columna tiene algún dato
            df_filtrado = df[df['Tipo'].notna() & (df['Tipo'] == 'Correctivo') & df[columna_nombre].notna() & df[columna_nombre].str.strip().astype(bool)]
            # Sumar la columna "Duración" filtrada, ignorando NaNs y valores no numéricos
            suma_duracion = df_filtrado['Duración'].dropna().sum()
            return suma_duracion if suma_duracion > 0 else "Sin datos 'Correctivo' con datos relevantes en columna"
        
        # Si la columna 'Tipo' existe pero no la columna relevante
        return "Columna relevante ('Nombre'/'codigo_personal_text') no encontrada"

    except Exception as e:
        return f"Error al procesar el archivo: {e}"
    
def duracion_tipo_correctivo_avance(nombre_archivo):
    try:
        # Cargar el archivo CSV
        df = pd.read_csv(os.path.join(ruta, nombre_archivo))
        # Reemplazar comas por puntos y convertir a flotante de forma segura
        df['Duración'] = pd.to_numeric(df['Duración'].str.replace(',', '.'), errors='coerce')

        # Verificar si la columna 'Tipo' existe y es adecuada para el filtrado
        if 'Tipo' in df.columns:
            # Filtrar filas donde la columna 'Tipo' es igual a 'Correctivo'
            df_filtrado = df[df['Tipo'].notna() & (df['Tipo'] == 'Correctivo')]
            # Calcular la suma de las duraciones para las filas filtradas
            suma_duracion = df_filtrado['Duración'].dropna().sum()
            # Obtener la cantidad de filas filtradas que contienen 'Correctivo'
            cantidad_filas_correctivo = len(df_filtrado)
            # Verificar si hay datos para retornar
            if cantidad_filas_correctivo > 0:
                # Calcular promedio de duración por cada fila 'Correctivo'
                promedio_duracion = suma_duracion / cantidad_filas_correctivo
                return promedio_duracion
            else:
                return "Sin datos 'Correctivo'"
        else:
            # Si la columna 'Tipo' no existe, retornar cero
            return 0
    except Exception as e:
        return f"Error: {e}"

    





# Extraer el código único de cuatro dígitos de cada archivo
codigos = [extraer_codigo(archivo) for archivo in archivos_csv]
# Calcular la sumatoria de duración para cada archivo
sumas_duracion = [sumar_duracion(archivo) for archivo in archivos_csv]
# Calcular la sumatoria de duración planeada para cada archivo
sumas_duracion_planeada = [sumar_duracion_planeada(archivo) for archivo in archivos_csv]
# Calcular la sumatoria de duración con avance para cada archivo
sumas_duracion_avance = [sumar_duracion_avance(archivo) for archivo in archivos_csv]
# Aplicar la función duracion_tipo_o_total a cada archivo CSV
duraciones_tipo_o_total = [duracion_tipo_o_total(archivo) for archivo in archivos_csv]
# Aplicar la función duracion_tipo_preventivo_o_total a cada archivo CSV
duraciones_tipo_preventivo_o_total = [duracion_tipo_preventivo_o_total(archivo) for archivo in archivos_csv]
# Aplicar la función duracion_tipo_preventivo_nombre_codigo a cada archivo CSV
duraciones_tipo_preventivo_nombre_codigo = [duracion_tipo_preventivo_nombre_codigo(archivo) for archivo in archivos_csv]
# Aplicar la función duracion_tipo_preventivo_avance a cada archivo CSV
duraciones_tipo_preventivo_avance = [duracion_tipo_preventivo_avance(archivo) for archivo in archivos_csv]
# Aplicar la función duracion_tipo_mantenimiento a cada archivo CSV
duraciones_tipo_predictivo_o_total = [duracion_tipo_predictivo_o_total(archivo) for archivo in archivos_csv]
# Aplicar la función duracion_tipo_predictivo_nombre_codigo a cada archivo CSV
duracion_tipo_predictivo_nombre_codigo = [duracion_tipo_predictivo_nombre_codigo(archivo) for archivo in archivos_csv]
# Aplicar la función duracion_tipo_predictivo_avance a cada archivo CSV
duracion_tipo_predictivo_avance = [duracion_tipo_predictivo_avance(archivo) for archivo in archivos_csv]
# Aplicar la función duracion_tipo_correctivo_o_total a cada archivo CSV
duraciones_tipo_correctivo_o_total = [duracion_tipo_correctivo_o_total(archivo) for archivo in archivos_csv]
# Aplicar la función duracion_tipo_correctivo_nombre_codigo a cada archivo CSV
duraciones_tipo_correctivo_nombre_codigo = [duracion_tipo_correctivo_nombre_codigo(archivo) for archivo in archivos_csv]
# Aplicar la función duracion_tipo_correctivo_avance a cada archivo CSV
duraciones_tipo_correctivo_avance = [duracion_tipo_correctivo_avance(archivo) for archivo in archivos_csv]


# Crear un DataFrame con los nombres de archivo, códigos y sumas de duración
df = pd.DataFrame({
    'Nombres de Archivo CSV': archivos_csv,
    'Código Único': codigos,
    'Sumatoria de Duración': sumas_duracion,
    'Sumatoria de Duración Planeada': sumas_duracion_planeada,
    'Sumatoria de Duración con Avance': sumas_duracion_avance,
    'Sumatoria de Duración (Tipo o Total)': duraciones_tipo_o_total,
    'Sumatoria de Duración (Preventivo o Total)': duraciones_tipo_preventivo_o_total,
    'Sumatoria de Duración (Preventivo con Nombre o Código)': duraciones_tipo_preventivo_nombre_codigo,
    'Sumatoria de Duración (Preventivo con Avance)': duraciones_tipo_preventivo_avance,
    'Sumatoria de Duración (predictivo o total)': duraciones_tipo_predictivo_o_total,
    'Sumatoria de Duración (predictivo con Nombre o Código)': duracion_tipo_predictivo_nombre_codigo,
    'Sumatoria de Duración (predictivo con Avance)': duracion_tipo_predictivo_avance,
    'Sumatoria de Duración (correctivo o total)': duraciones_tipo_correctivo_o_total,
    'Sumatoria de Duración (correctivo con Nombre o Código)': duraciones_tipo_correctivo_nombre_codigo,
    'Sumatoria de Duración (correctivo con Avance)': duraciones_tipo_correctivo_avance
    
})

# Convertir la columna 'Sumatoria de Duración con Avance' a tipo numérico
df['Sumatoria de Duración con Avance'] = pd.to_numeric(df['Sumatoria de Duración con Avance'], errors='coerce')

# Calcular el porcentaje y agregarlo como una nueva columna
df['Porcentaje'] = (df['Sumatoria de Duración con Avance'] / df['Sumatoria de Duración'] * 100).map(lambda x: f"{x:.2f}%")

# Guardar el DataFrame en un archivo Excel
nombre_excel = "Lista_Archivos_CSV_con_Codigos_y_Duraciones.xlsx"
ruta_completa_excel = os.path.join(ruta, nombre_excel)
df.to_excel(ruta_completa_excel, index=False)

print(f"Archivo Excel creado con éxito en: {ruta_completa_excel}")



Archivo Excel creado con éxito en: C:\Users\cesar\Desktop\BIBLIOCASTIA\TRANFORMACION CSV A XLSX\Lista_Archivos_CSV_con_Codigos_y_Duraciones.xlsx


In [ ]:
def unify_data(file_path):
    # Carga el libro de trabajo existente
    wb = load_workbook(file_path)
    ws = wb['Sheet1']  # Asume que los datos están en la hoja llamada 'Sheet1'

    # Lee los datos desde la hoja específica
    data = pd.read_excel(file_path, sheet_name='Sheet1')
    
    # Convierte las columnas de duración a numéricas, gestionando errores
    for col in ['Sumatoria de Duración', 'Sumatoria de Duración Planeada', 'Sumatoria de Duración con Avance']:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Agrupa por 'Código Único' y agrega datos
    unified_df = data.groupby('Código Único').agg({
        'Sumatoria de Duración': 'sum',
        'Sumatoria de Duración Planeada': 'sum',
        'Sumatoria de Duración con Avance': 'sum'
    }).reset_index()

    # Calcula el porcentaje y añade como nueva columna
    unified_df['Porcentaje'] = (unified_df['Sumatoria de Duración con Avance'] / unified_df['Sumatoria de Duración'] * 100).map(lambda x: f"{x:.2f}%")

    # Agrega una nueva hoja al libro con los datos unificados
    new_ws = wb.create_sheet('indicadores_cam')
    for r in dataframe_to_rows(unified_df, index=False, header=True):
        new_ws.append(r)

    # Aplica el formato de color gradiente
    col_letter = get_column_letter(unified_df.columns.get_loc('Porcentaje') + 1)
    for row in range(2, new_ws.max_row + 1):
        cell = new_ws[f'{col_letter}{row}']
        try:
            percentage = float(cell.value.strip('%'))
            green_intensity = int((percentage / 100) * 255)
            red_intensity = 255 - green_intensity
            hex_color = f"{red_intensity:02X}{green_intensity:02X}00"
            cell.fill = PatternFill(start_color=hex_color, end_color=hex_color, fill_type="solid")
        except (ValueError, AttributeError):  # Maneja también AttributeError por si 'cell.value' es None
            continue

    # Guarda los cambios en el archivo de Excel existente
    wb.save(file_path)

# Especifica la ruta al archivo de Excel
file_path = ruta_archivo_entregable

# Llama a la función
unify_data(file_path)

print("Los datos han sido unificados y añadidos a la nueva hoja 'indicadores_cam' con formato de color.")


Los datos han sido unificados y añadidos a la nueva hoja 'indicadores_cam' con formato de color.


In [ ]:
def unify_data(file_path):
    # Carga el libro de trabajo existente
    wb = load_workbook(file_path)
    ws = wb['Sheet1']  # Asume que los datos están en la hoja llamada 'Sheet1'

    # Lee los datos desde la hoja específica
    data = pd.read_excel(file_path, sheet_name='Sheet1')
    
    # Convierte las columnas de duración a numéricas, gestionando errores
    col_names = [
        'Sumatoria de Duración (Preventivo o Total)', 
        'Sumatoria de Duración (Preventivo con Nombre o Código)', 
        'Sumatoria de Duración (Preventivo con Avance)'
    ]
    for col in col_names:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Agrupa por 'Código Único' y agrega datos
    unified_df = data.groupby('Código Único').agg({
        'Sumatoria de Duración (Preventivo o Total)': 'sum',
        'Sumatoria de Duración (Preventivo con Nombre o Código)': 'sum',
        'Sumatoria de Duración (Preventivo con Avance)': 'sum'
    }).reset_index()

    # Calcula el porcentaje y añade como nueva columna
    unified_df['Porcentaje'] = (unified_df['Sumatoria de Duración (Preventivo con Avance)'] / unified_df['Sumatoria de Duración (Preventivo o Total)'] * 100).map(lambda x: f"{x:.2f}%")

    # Agrega una nueva hoja al libro con los datos unificados
    new_ws = wb.create_sheet('indicadores_cam_Preventivo')
    for r in dataframe_to_rows(unified_df, index=False, header=True):
        new_ws.append(r)

    # Aplica el formato de color gradiente a la columna 'Porcentaje'
    col_letter = get_column_letter(unified_df.columns.get_loc('Porcentaje') + 1)
    for row in range(2, new_ws.max_row + 1):
        cell = new_ws[f'{col_letter}{row}']
        try:
            percentage = float(cell.value.strip('%'))
            green_intensity = int((percentage / 100) * 255)
            red_intensity = 255 - green_intensity
            hex_color = f"{red_intensity:02X}{green_intensity:02X}00"
            cell.fill = PatternFill(start_color=hex_color, end_color=hex_color, fill_type="solid")
        except (ValueError, AttributeError):  # Maneja también AttributeError por si 'cell.value' es None
            continue

    # Guarda los cambios en el archivo de Excel existente
    wb.save(file_path)

# Especifica la ruta al archivo de Excel
file_path = ruta_archivo_entregable

# Llama a la función
unify_data(file_path)

print("Los datos han sido unificados y añadidos a la nueva hoja 'indicadores_cam_Preventivo', incluyendo el cálculo de porcentaje con formato de color.")


Los datos han sido unificados y añadidos a la nueva hoja 'indicadores_cam_Preventivo', incluyendo el cálculo de porcentaje con formato de color.


In [ ]:
def unify_data(file_path):
    # Carga el libro de trabajo existente
    wb = load_workbook(file_path)
    ws = wb['Sheet1']  # Asume que los datos están en la hoja llamada 'Sheet1'

    # Lee los datos desde la hoja específica
    data = pd.read_excel(file_path, sheet_name='Sheet1')
    
    # Convierte las columnas de duración a numéricas, gestionando errores
    col_names = [
        'Sumatoria de Duración (predictivo o total)', 
        'Sumatoria de Duración (predictivo con Nombre o Código)', 
        'Sumatoria de Duración (predictivo con Avance)'
    ]
    for col in col_names:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Agrupa por 'Código Único' y agrega datos
    unified_df = data.groupby('Código Único').agg({
        'Sumatoria de Duración (predictivo o total)': 'sum',
        'Sumatoria de Duración (predictivo con Nombre o Código)': 'sum',
        'Sumatoria de Duración (predictivo con Avance)': 'sum'
    }).reset_index()

    # Calcula el porcentaje y añade como nueva columna
    unified_df['Porcentaje'] = (unified_df['Sumatoria de Duración (predictivo con Avance)'] / unified_df['Sumatoria de Duración (predictivo o total)'] * 100).map(lambda x: f"{x:.2f}%")

    # Agrega una nueva hoja al libro con los datos unificados
    new_ws = wb.create_sheet('indicadores_cam_Predictivo')
    for r in dataframe_to_rows(unified_df, index=False, header=True):
        new_ws.append(r)

    # Aplica el formato de color gradiente a la columna 'Porcentaje'
    col_letter = get_column_letter(unified_df.columns.get_loc('Porcentaje') + 1)
    for row in range(2, new_ws.max_row + 1):
        cell = new_ws[f'{col_letter}{row}']
        try:
            percentage = float(cell.value.strip('%'))
            green_intensity = int((percentage / 100) * 255)
            red_intensity = 255 - green_intensity
            hex_color = f"{red_intensity:02X}{green_intensity:02X}00"
            cell.fill = PatternFill(start_color=hex_color, end_color=hex_color, fill_type="solid")
        except (ValueError, AttributeError):  # Maneja también AttributeError por si 'cell.value' es None
            continue

    # Guarda los cambios en el archivo de Excel existente
    wb.save(file_path)

# Especifica la ruta al archivo de Excel
file_path = ruta_archivo_entregable

# Llama a la función
unify_data(file_path)

print("Los datos han sido unificados y añadidos a la nueva hoja 'indicadores_cam_Predictivo', incluyendo el cálculo de porcentaje con formato de color.")


Los datos han sido unificados y añadidos a la nueva hoja 'indicadores_cam_Predictivo', incluyendo el cálculo de porcentaje con formato de color.


In [ ]:
def unify_data(file_path):
    # Carga el libro de trabajo existente
    wb = load_workbook(file_path)
    ws = wb['Sheet1']  # Asume que los datos están en la hoja llamada 'Sheet1'

    # Lee los datos desde la hoja específica
    data = pd.read_excel(file_path, sheet_name='Sheet1')
    
    # Convierte las columnas de duración a numéricas, gestionando errores
    col_names = [
        'Sumatoria de Duración (correctivo o total)', 
        'Sumatoria de Duración (correctivo con Nombre o Código)', 
        'Sumatoria de Duración (correctivo con Avance)'
    ]
    for col in col_names:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Agrupa por 'Código Único' y agrega datos
    unified_df = data.groupby('Código Único').agg({
        'Sumatoria de Duración (correctivo o total)': 'sum',
        'Sumatoria de Duración (correctivo con Nombre o Código)': 'sum',
        'Sumatoria de Duración (correctivo con Avance)': 'sum'
    }).reset_index()

    # Calcula el porcentaje y añade como nueva columna
    unified_df['Porcentaje'] = (unified_df['Sumatoria de Duración (correctivo con Avance)'] / unified_df['Sumatoria de Duración (correctivo o total)'] * 100).map(lambda x: f"{x:.2f}%")

    # Agrega una nueva hoja al libro con los datos unificados
    new_ws = wb.create_sheet('indicadores_cam_Correctivo')
    for r in dataframe_to_rows(unified_df, index=False, header=True):
        new_ws.append(r)

    # Aplica el formato de color gradiente a la columna 'Porcentaje'
    col_letter = get_column_letter(unified_df.columns.get_loc('Porcentaje') + 1)
    for row in range(2, new_ws.max_row + 1):
        cell = new_ws[f'{col_letter}{row}']
        try:
            percentage = float(cell.value.strip('%'))
            green_intensity = int((percentage / 100) * 255)
            red_intensity = 255 - green_intensity
            hex_color = f"{red_intensity:02X}{green_intensity:02X}00"
            cell.fill = PatternFill(start_color=hex_color, end_color=hex_color, fill_type="solid")
        except (ValueError, AttributeError):  # Maneja también AttributeError por si 'cell.value' es None
            continue

    # Guarda los cambios en el archivo de Excel existente
    wb.save(file_path)

# Especifica la ruta al archivo de Excel
file_path = ruta_archivo_entregable

# Llama a la función
unify_data(file_path)

print("Los datos han sido unificados y añadidos a la nueva hoja 'indicadores_cam_Correctivo', incluyendo el cálculo de porcentaje con formato de color.")


Los datos han sido unificados y añadidos a la nueva hoja 'indicadores_cam_Correctivo', incluyendo el cálculo de porcentaje con formato de color.


In [ ]:
import os
import pandas as pd
import re  # Módulo para trabajar con expresiones regulares

def limpiar_nombre(nombre):
    # Elimina el texto dentro de los paréntesis incluyendo los paréntesis mismos
    return re.sub(r" ?\([^)]+\)", "", nombre)

# Listar todos los archivos en el directorio
archivos = os.listdir(ruta)

print("Procesando archivos CSV:")
for archivo in archivos:
    if archivo.endswith('.csv'):
        # Limpiar el nombre del archivo
        nombre_limpio = limpiar_nombre(archivo)
        # Construir la ruta completa al archivo original y al archivo limpio
        archivo_completo = os.path.join(ruta, archivo)
        archivo_limpio_completo = os.path.join(ruta, nombre_limpio)
        
        # Leer el archivo CSV
        df = pd.read_csv(archivo_completo, encoding='utf-8')
        
        # Mostrar la cantidad de filas y el nombre limpio
        print(f"El archivo {nombre_limpio} tiene {df.shape[0]} filas.")
        
        # Reemplazar puntos por comas en la columna 'Duración' si existe
        if 'Duración' in df.columns:
            df['Duración'] = df['Duración'].astype(str).str.replace('.', ',')
        
        # Guardar el archivo con los cambios bajo el nuevo nombre limpio
        df.to_csv(archivo_limpio_completo, index=False, encoding='utf-8')
        
        # Si el nombre se ha cambiado, podemos optar por eliminar el archivo original
        if nombre_limpio != archivo:
            os.remove(archivo_completo)
        
        print(f"Archivo procesado y limpio: {nombre_limpio}")


Procesando archivos CSV:
El archivo ACTIVIDADES 0002_GASCOL CENTRO.csv tiene 57 filas.
Archivo procesado y limpio: ACTIVIDADES 0002_GASCOL CENTRO.csv
El archivo ACTIVIDADES 0011_GASCOL SUR.csv tiene 310 filas.
Archivo procesado y limpio: ACTIVIDADES 0011_GASCOL SUR.csv
El archivo ACTIVIDADES 0021_G. LUX BOGOTÁ.csv tiene 693 filas.
Archivo procesado y limpio: ACTIVIDADES 0021_G. LUX BOGOTÁ.csv
El archivo ACTIVIDADES 0079_BELLO.csv tiene 718 filas.
Archivo procesado y limpio: ACTIVIDADES 0079_BELLO.csv
El archivo ACTIVIDADES 0092_MEDELLÍN.csv tiene 260 filas.
Archivo procesado y limpio: ACTIVIDADES 0092_MEDELLÍN.csv
El archivo ACTIVIDADES 0111_MALAMBO.csv tiene 875 filas.
Archivo procesado y limpio: ACTIVIDADES 0111_MALAMBO.csv
El archivo ACTIVIDADES 0198_POSTOBÓN PEREIRA.csv tiene 64 filas.
Archivo procesado y limpio: ACTIVIDADES 0198_POSTOBÓN PEREIRA.csv
El archivo ACTIVIDADES 0381_G. LUX MONTERÍA.csv tiene 64 filas.
Archivo procesado y limpio: ACTIVIDADES 0381_G. LUX MONTERÍA.csv
El a

In [ ]:
import os

# Define la ruta donde se encuentran los archivos
ruta = r"C:\Users\cesar\Desktop\BIBLIOCASTIA\TRANFORMACION CSV A XLSX"

# Función para contar archivos que contienen una palabra específica en el nombre
def contar_archivos(ruta, palabra_clave):
    contador = 0
    # Listar todos los archivos en la ruta
    for archivo in os.listdir(ruta):
        # Checar si el archivo es un .csv y contiene la palabra clave
        if archivo.endswith('.csv') and palabra_clave in archivo:
            contador += 1
    return contador

# Contar archivos que contienen 'actividades' y 'inspecciones' en el nombre
cantidad_actividades = contar_archivos(ruta, 'ACTIVIDADES')
cantidad_inspecciones = contar_archivos(ruta, 'INSPECCIONES')

print(f"Cantidad de archivos .csv que contienen 'actividades': {cantidad_actividades}")
print(f"Cantidad de archivos .csv que contienen 'inspecciones': {cantidad_inspecciones}")


Cantidad de archivos .csv que contienen 'actividades': 18
Cantidad de archivos .csv que contienen 'inspecciones': 18


In [ ]:
import os
import pandas as pd

# Define la ruta donde se encuentran los archivos
ruta = r"C:\Users\cesar\Desktop\BIBLIOCASTIA\TRANFORMACION CSV A XLSX"

# Función para unificar archivos que contienen una palabra específica en el nombre
def unificar_archivos(ruta, palabra_clave, nombre_archivo_salida):
    frames = []  # Lista para guardar los dataframes temporales
    # Listar todos los archivos en la ruta
    for archivo in os.listdir(ruta):
        # Checar si el archivo es un .csv y contiene la palabra clave
        if archivo.endswith('.csv') and palabra_clave in archivo.upper():
            # Leer el archivo CSV y agregarlo a la lista de frames
            df = pd.read_csv(os.path.join(ruta, archivo))
            frames.append(df)
    # Concatenar todos los dataframes en uno solo
    df_concatenado = pd.concat(frames, ignore_index=True)
    # Guardar el dataframe combinado en un nuevo archivo CSV
    df_concatenado.to_csv(os.path.join(ruta, nombre_archivo_salida), index=False)
    return len(frames)

# Unificar archivos de actividades
cantidad_archivos_actividades = unificar_archivos(ruta, 'ACTIVIDADES', 'actividades_unificadas.csv')
print(f"Se han unificado {cantidad_archivos_actividades} archivos de 'actividades'.")

# Unificar archivos de inspecciones
cantidad_archivos_inspecciones = unificar_archivos(ruta, 'INSPECCIONES', 'inspecciones_unificadas.csv')
print(f"Se han unificado {cantidad_archivos_inspecciones} archivos de 'inspecciones'.")



Se han unificado 18 archivos de 'actividades'.
Se han unificado 18 archivos de 'inspecciones'.
